# Setting up a data processing worflow for NetCDF in WINGS

This Jupyter Notebook walks through the necessary steps to set up a data processing workflow for NetCDF files to files needed by [TOPOFLOW](https://github.com/peckhams/topoflow) in [WINGS](http://www.wings-workflows.org). The data used in this example and necessary processing steps are described [here](https://github.com/khider/netCDFTutorial/blob/master/TOPOFLOW%20var%20example.ipynb).

This Notebook is set up so that each cell represents a workflow component. The text lists the necessary data and parameters inputs for each of the component, the outputs, and the assumptions.

Table of Contents
1. [openNetCDF](#opennetcdf)
2. [adjustTime](#adjusttime)
3. [selectVar](#selectVar)
4. [calculateRH](#calculateRH)
5. [calculateWind](#calculateWind)
6. [amountToRate](#amount)
7. [completeCheck](#complete)
8. [adjustUnits](#units)
9. [adjustformat](#format)

## <a name='opennetcdf'> openNetCDF </a>

This component open the NetCDF file and dumps the content into a Python dictionary, exposing the metadata in the process.

- Inputs: NetCDF files
- Outputs: Pickled file
- Parameters: None

In [1]:
from netCDF4 import MFDataset
import pickle

def getKeys(nc_fid):
    '''Get the variables names in the NetCDF file'''
    keys=[]
    nc_vars = [var for var in nc_fid.variables]
    for vars in nc_vars:
        keys.append(getattr(nc_fid.variables[vars],'long_name'))
    return keys
        
def getMFNcVar(nc_fid, keys):
    ''' Extract variables from a dataset across multiple netCDF files.
    
    This function gets the variable contained in a netCDF file 
    and return them into Python nested dictionaries. The first
    dictionary's key contains the longname, while the
    second dictionary contains values, standard name (CF),
    units and the missing data flag.
    
    Args:
        nc_files (list): A list of netCDF files containing the dataset
        keys (list): A list of keys to fetch the variables according
            to the CF standard
    
    Returns:
        dict_out (dict): A dictionary containing the standard names as keys and
            the associated data as values.
    '''
    
    nc_vars = [var for var in nc_fid.variables]
    
    #Make empty lists to collect the info
    #longname (should be using the CF conventions)
    nc_vars_longname=[]
    #Units
    nc_vars_units=[]
    # Get the standard name
    nc_vars_standardname=[]
    #Corrections
    nc_vars_scale_factor=[]
    nc_vars_add_offset=[]
    #Missing values
    nc_vars_missing_value=[]
    
    for vars in nc_vars:
        if 'long_name' in nc_fid.variables[vars].ncattrs():
            nc_vars_longname.append(getattr(nc_fid.variables[vars],'long_name'))
        else:
            nc_vars_longname.append(vars)
        if 'units' in nc_fid.variables[vars].ncattrs():
            nc_vars_units.append(getattr(nc_fid.variables[vars],'units'))
        else:
            nc_vars_units.append('NA')
        if 'standard_name' in nc_fid.variables[vars].ncattrs():
            nc_vars_standardname.append(getattr(nc_fid.variables[vars],'standard_name'))
        else:
            nc_vars_standardname.append("NA")    
        if 'scale_factor' in nc_fid.variables[vars].ncattrs():
            nc_vars_scale_factor.append(getattr(nc_fid.variables[vars],'scale_factor'))
        else:
            nc_vars_scale_factor.append(1)
        if 'add_offset' in nc_fid.variables[vars].ncattrs():
            nc_vars_add_offset.append(getattr(nc_fid.variables[vars],'add_offset'))
        else:
            nc_vars_add_offset.append(0) 
        if 'missing_value' in nc_fid.variables[vars].ncattrs(): 
            nc_vars_missing_value.append(getattr(nc_fid.variables[vars],'missing_value'))
        else:
            nc_vars_missing_value.append('NA')
    # Check for the list against the desired variables and output.
    dict_out ={}
    for name in nc_vars_longname:
        if name in keys:
            f = {'values':[],'units':[],'missing_value':[], 'standard_name':{}}
            idx = nc_vars_longname.index(name)
            f['values']=(nc_fid.variables[nc_vars[idx]][:]*nc_vars_scale_factor[idx])\
                +nc_vars_add_offset[idx]
            f['units']=nc_vars_units[idx]
            f['missing_value'] = nc_vars_missing_value[idx]
            f['standard_name'] = nc_vars_standardname[idx]
            dict_out[name] = f
    
    return dict_out

root = "/Volumes/Data HD/Documents/MINT/WINGS/NetCDFWings"
files = ["Oct2010.nc","Nov2010.nc","Dec2010.nc"]

file_names =[]
for name in files:
    file_names.append(root+"/"+name)
    
#Open the file and get the keys for this example
nc_fid = MFDataset(file_names)
keys = getKeys(nc_fid)
dict_out=getMFNcVar(nc_fid,keys)

with open("pickle1.p","wb") as handle:
    pickle.dump(dict_out, handle, protocol=pickle.HIGHEST_PROTOCOL)

## <a name='adjusttime'> adjustTime </a>

Time coordinate values pose a special challeneg to netCDF users. Most metadata standards (such as CF) specify that time should be measured relative to a fixed data using a certain calendar, with units specified like **hours since YY-MM-DD hh:mm:ss**. These units can be awkward to deal with, without a utility to convert the valuesto and from calendar dates. The function called `num2date` and `date2num` are provided with the `netCDF4` package to do just that.

`num2date` converts numeric values of time in the specified **units** and **calendar** to datetime objects, and `date2num` does the reverse.

- `units`: a string of the form **&lt;time units&gt; since &lt;reference time&gt;** describing the time units. **&lttime units&gh;** can be days, hours, minutes, seconds, milliseconds, or microseconds. **&lt;reference time&gt;** is the time origin.
- `calendar`: describes the calendar used in the time calculations. All the values currently defined in the [CF metadata convention](http://cfconventions.org/) are valid calendars: **'standard'**, **'gregorian'**, **'proleptic_gregorian'**, **'noleap'**, **'365_day'**, **'360_day'**, **'julian'**, **'all_leap'**, **'366_day'**. Default is **'standard'**, which is a mixed Julian/Gregorian calendar.

([source](http://unidata.github.io/netcdf4-python/#section7))

- Inputs: pickle file
- Outputs: pickle file
- parameters: one of the valid calendars described above.

In [3]:
param = 'standard'

import pickle
from datetime import datetime, timedelta
from netCDF4 import num2date, date2num

with open("pickle1.p", 'rb') as handle:
    dict_out = pickle.load(handle)

dates = num2date(dict_out['time']['values'][:],dict_out['time']['units'])
dict_out['dates']={'values':dates, 'calendar':param, 'units':'NA'}

with open("pickle2.p","wb") as handle:
    pickle.dump(dict_out, handle, protocol=pickle.HIGHEST_PROTOCOL)

## <a name='selectVar'> selectVar </a>

This component selects the variables in the dictionary that correspond to the model variables. If model variables are not present, send a warning.

- Inputs: pickle file from previous step, a text file containing the list of variables
- Outputs: warning file. This file will be needed for 
- Parameters: none

In [4]:
import pickle

f2 = open('modelVar.txt','r') 
var = f2.readlines()
var = [x.strip() for x in var]

with open("pickle2.p", 'rb') as handle:
    dict_out = pickle.load(handle)

warningsVar = []
for item in var:
    if item not in dict_out.keys():
       warningsVar.append(item)

f3 = open('warnings.txt','w')
for item in warningsVar:
   f3.write("%s\n" % item)            
f3.close()   

## <a name='calculateRH'> calculateRH </a>

This functions calculates relative humidity from temperature and dew point temperature. Note that one would have to know to use the temperature at the same atmospheric level (2m here) for the calculation.

Relative humidity can be calculate from temperature and dewpoint temperature. In the query, care should be taken as to choose temperature taken at the same elevation. In this particular example, we use `2 metre temperature` and `2 metre dewpoint temperature`.

Relative humidity can then be calculated as follows:

$RH=100\times\frac{e^{\frac{17.625\times TD}{243.04+TD}}}{e^{\frac{17.625\times T}{243.04+T}}}$

where T (temperature) and TD (dewpoint temperature) are in deg C.

- Input: pickle file
- Output: pickle file
- Parameters: None

In [5]:
import pickle
import numpy as np

with open("pickle2.p", 'rb') as handle:
    dict_out = pickle.load(handle)
    
TD = np.array(dict_out['2 metre dewpoint temperature']['values'])
T = np.array(dict_out['2 metre temperature']['values'])
RH = 100*(np.exp((17.625*TD)/(243.04+TD))/np.exp((17.625*T)/(243.04+T)))

dict_out['Relative Humidity']={'values':RH,'units':'NA'} 

with open("pickle3.p","wb") as handle:
    pickle.dump(dict_out, handle, protocol=pickle.HIGHEST_PROTOCOL)

## <a name='calculatewind'> calculateWind </a>

Wind speed can be calculated from the u-wind and v-wind components. TOPOFLOW also calls for the reference height which is contained in the name of the variables: `10 metre U wind component` and `10 metre V wind component`. As for relative humidity, one would have to use wind components at the same reference height (here 10m). The reference height is an input to TOPOFLOW and needs to be included in the dictionary.

Wind speed can be calculated as follows:

$WS = \sqrt{U^{2}+V^{2}}$

- Input: pickle file
- Output: pickle file
- Parameters: None

In [6]:
import pickle
import numpy as np
import re

with open("pickle3.p", 'rb') as handle:
    dict_out = pickle.load(handle)
    
U = np.array(dict_out['10 metre U wind component']['values'])
V = np.array(dict_out['10 metre V wind component']['values'])

W = np.sqrt(U**2+V**2)

dict_out['Wind Speed']={'values':W,'units':dict_out['10 metre V wind component']['units']}

#Get the wind height
s = '10 metre U wind component'
m = re.findall(r'\d+', s)[0]
dict_out['Wind reference height']={'values':int(m),'units':dict_out['10 metre V wind component']['units']}

with open("pickle4.p","wb") as handle:
   pickle.dump(dict_out, handle, protocol=pickle.HIGHEST_PROTOCOL)

## <a name='amount'> amountToRate </a>

Precipitation is given as total amount in the reanalysis dataset. TOPOFLOW takes a rate input of mm/hr, which requires knowing the time step of the data (in this case, 3hr). The time step would be given upon dowload and should be stored in the data catalog. Assumed known in this example.

- Input: pickle file
- Output: pickle file
- Parameters: None

In [7]:
import pickle
import numpy as np

with open("pickle4.p", 'rb') as handle:
    dict_out = pickle.load(handle)
    
precip_rate = 1000*np.array(dict_out['Total precipitation']['values'])/3

dict_out['Precipitation rate'] = {'values':precip_rate.tolist(), \
                                  'units':'mm/hr','notes':\
                                  'converted from amount, Total precipitation'}

with open("pickle5.p","wb") as handle:
    pickle.dump(dict_out, handle, protocol=pickle.HIGHEST_PROTOCOL)

## <a name='complete'> completeCheck </a>

This component ensures that all the proper variables are present in the pickled file. Will raise a `SystemExit` error if not every variable is present.

- Input: pickle file
- Output: None
- Parameters: None

In [11]:
import pickle
import sys

with open("pickle5.p", 'rb') as handle:
    dict_out = pickle.load(handle)
    
f2 = open(root+'/modelVar.txt','r') 
var = f2.readlines()
var = [x.strip() for x in var]

for item in var:
    if item not in dict_out.keys():
        sys.exit('variables are missing')

## <a name='units'> adjustUnits </a>

This component adjust the units. We will need to have a library of units transformation. For this example, the units for the modelVarUnits file were made to match that present in the data file except for pressure. Note that these units do not correspond to the ones in Topoflow (especially regarding temperature).

- Inputs: Pickled file
- Outputs: Pickled file
- Parameters: None

In [12]:
import pickle

with open("pickle5.p", 'rb') as handle:
    dict_out = pickle.load(handle)
    
f2 = open('modelVar.txt','r') 
var = f2.readlines()
var = [x.strip() for x in var]

f3 =  open('modelVarUnits.txt','r')
units = f3.readlines()
units = [x.strip() for x in units]

for idx, item in enumerate(var):
    dataUnits = dict_out[item]['units']
    if dataUnits != units[idx]:
        print("Adjusting "+item+' units...')
        P = dict_out['Surface pressure']['values']/100
        dict_out['Surface pressure']={'values':P,'units':'mbar','notes':'units have been converted'}
        
with open("pickle6.p","wb") as handle:
    pickle.dump(dict_out, handle, protocol=pickle.HIGHEST_PROTOCOL)        

Adjusting Surface pressure units...


## <a name=format> adjustFormat </a>

This component produces the binary text files for use by Topoflow.

- Input: Pickled file
- Outputs: Collection of text files
- Parameters: None

In [15]:
import pickle
import numpy as np

with open("pickle6.p", 'rb') as handle:
    dict_out = pickle.load(handle)

f2 = open('modelVar.txt','r') 
var = f2.readlines()
var = [x.strip() for x in var]

for item in var:
    dataValues = dict_out[item]['values']
    dataValues = np.float32(dataValues)
    filename = item+'.txt'
    f = open(filename,'wb')
    #dataValues.byteswap(True)
    dataValues.tofile(f)
    f.close()